In [1]:
import cv2
import time
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose


In [2]:
def elbow_angle(a, b, c):
    radians = np.arctan2(c.y-b.y, c.x-b.x) - np.arctan2(a.y-b.y, a.x-b.x)
    angle = np.abs(radians*180/np.pi)

    if angle > 180:
        angle = 360 - angle

    return round(angle, 2)

In [ ]:
def torso_angle(a, b, c, d):
    """
    Calculates the angle between torso and the Z-axis.

    Args:
        p1 (tuple/list): middle point of shoulders
        p2 (tuple/list): middel point of hips

    Returns:
        float: Angle in degrees between the line and the positive Z-axis.
    """
    p1 = ((a.x+b.x)/2, (a.y+b.y)/2, (a.z+b.z)/2)  # Midpoint of a and b
    p2 = ((c.x+d.x)/2, (c.y+d.y)/2, (c.z+d.z)/2)  # Midpoint of c and d
    
    # 1. Define Z-axis unit vector
    z_axis_vector = np.array([0, 0, 1])

    # 2. Calculate the line vector (P2 - P1)
    line_vector = np.array(p2) - np.array(p1)

    # 3. Calculate the dot product
    dot_product = np.dot(line_vector, z_axis_vector)

    # 4. Calculate magnitudes
    magnitude_line = np.linalg.norm(line_vector)
    magnitude_z = np.linalg.norm(z_axis_vector) # This is just 1

    # Avoid division by zero if the line is a point
    if magnitude_line == 0:
        return 0.0 # Or handle as error

    # 5. Calculate the cosine of the angle
    # Clamp the value to [-1, 1] to avoid floating point errors with acos
    cos_theta = np.clip(dot_product / (magnitude_line * magnitude_z), -1.0, 1.0)

    # 6. Calculate the angle in radians and convert to degrees
    angle_radians = np.arccos(cos_theta)
    angle = np.degrees(angle_radians)

    if angle > 180:
        angle = 360 - angle


    return round(angle, 2)



1. Torso Angle:

    less than 20° relative to the vertical direction (Z axis)

2. Elbow Angle:

    At the top:
    greater than 150 

    At the bottom:
    less than 90° 


In [ ]:
cap = cv2.VideoCapture(0)

start_time = time.time()
counter = 0
rate = 0
rate_t = "0.0"
stage = None

with mp_pose.Pose(
    min_detection_confidence=0.5, 
    min_tracking_confidence=0.5
) as pose:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark
            l_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]
            l_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
            l_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
            l_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
 
            r_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]
            r_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value]
            r_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
            r_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
 
            lh_angle = elbow_angle(l_wrist, l_elbow, l_shoulder)
            rh_angle = elbow_angle(r_wrist, r_elbow, r_shoulder)
            t_angle = torso_angle(l_shoulder, r_shoulder, l_hip, r_hip)

            cv2.putText(image,
                        str(lh_angle),
                        tuple(np.multiply((l_elbow.x, l_elbow.y), [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.5,
                        (255, 255, 0),
                        2,
                        cv2.LINE_AA
                        )
            cv2.putText(image,
                        str(rh_angle),
                        tuple(np.multiply((r_elbow.x, r_elbow.y), [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.5,
                        (255, 255, 0),
                        2,
                        cv2.LINE_AA
                        )

            cv2.putText(image,
                        str(t_angle),
                        tuple(np.multiply(((l_shoulder.x+r_shoulder.x)/2, (l_shoulder.y+r_shoulder.y)/2), [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.5,
                        (255, 255, 0),
                        2,
                        cv2.LINE_AA
                        )
            cv2.putText(image,
                        timer_text,
                        (10, 470),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.7,
                        (0, 255, 0),
                        2,
                        cv2.LINE_AA
                        )

            # Curl counter logic
            if lh_angle > 150 and rh_angle > 150:
                stage = 'up'
            if lh_angle < 90 and rh_angle < 90 and (t_angle - 90) < 20 and stage == 'up':
                stage = 'down'
                counter += 1

        except:
            pass
        
        cv2.rectangle(image, (0,0), (150,50), (255,100,0), -1)
        cv2.putText(image, 'REPS: ',
                    (0,35),
                    cv2.FONT_HERSHEY_COMPLEX,
                    0.7,
                    (0,255,255),
                    1,
                    cv2.LINE_AA)
        cv2.putText(image,
                    str(counter),
                    (80,40),
                     cv2.FONT_HERSHEY_COMPLEX,
                    1.0,
                    (255,255,255),
                    2,
                    cv2.LINE_AA)

        cv2.rectangle(image, (440,0), (620,50), (255,100,0), -1)
        cv2.putText(image, 'Stage: ',
                    (440,35),
                    cv2.FONT_HERSHEY_COMPLEX,
                    0.7,
                    (0,255,255),
                    1,
                    cv2.LINE_AA)
        cv2.putText(image,
                    str(stage),
                    (530,40),
                     cv2.FONT_HERSHEY_COMPLEX,
                    1.0,
                    (255,255,255),
                    2,
                    cv2.LINE_AA)

        cv2.rectangle(image, (220,0), (380,50), (255,100,0), -1)
        elapsed_time = time.time() - start_time
        minutes = int(elapsed_time // 60)
        seconds = int(elapsed_time % 60)
        timer_text = f"{minutes:02d}:{seconds:02d}"
        rate = counter / (minutes * 60 + seconds + 0.001) * 60
        rate_t = f"{rate:.1f}"
        cv2.putText(image, '#/min: ',
                    (220,35),
                    cv2.FONT_HERSHEY_COMPLEX,
                    0.7,
                    (0,255,255),
                    1,
                    cv2.LINE_AA)
        cv2.putText(image,
                    str(rate_t),
                    (320,40),
                     cv2.FONT_HERSHEY_COMPLEX,
                    1.0,
                    (255,255,255),
                    2,
                    cv2.LINE_AA)

        # Draw landmarks
        mp_drawing.draw_landmarks(
            image, 
            results.pose_landmarks, 
            mp_pose.POSE_CONNECTIONS
        )
        
        cv2.imshow("Pose", image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()